In [ ]:
import numpy as np
import re

def calculate_average_bstrength(trial_range, filepath='home/folder/gooddata.txt'):
    """
    Calculate average of last 10 B_strength values for given trial range
    
    Args:
        trial_range (str): Range in format 'start-end' (e.g., '20-25')
        filepath (str): Path to data file
    
    Returns:
        dict: Averages for each trial in the range
    """
    # Parse trial range
    start, end = map(int, trial_range.split('-'))
    trial_numbers = range(start, end + 1)
    
    results = {}
    
    try:
        with open(filepath, 'r') as f:
            lines = f.readlines()
            
            for trial in trial_numbers:
                # Find all data blocks for this trial
                pattern = re.compile(rf"# B_strength\[:, {trial}\]")
                matches = [i for i, line in enumerate(lines) if pattern.search(line)]
                
                if not matches:
                    print(f"Warning: Trial {trial} not found in file")
                    continue
                
                # Get the last block for this trial (in case of multiple entries)
                last_block_start = matches[-1]
                
                # Find where the data ends (next header or EOF)
                data_end = len(lines)
                for i in range(last_block_start + 1, len(lines)):
                    if lines[i].startswith('# B_strength'):
                        data_end = i
                        break
                
                # Extract last 10 values
                data_lines = lines[last_block_start + 1 : data_end]
                values = [float(line.strip()) for line in data_lines[-10:]]
                
                if len(values) < 10:
                    print(f"Warning: Trial {trial} has only {len(values)} values")
                
                avg = np.mean(values)
                results[trial] = avg
                
    except FileNotFoundError:
        print(f"Error: File not found at {filepath}")
        return None
    except Exception as e:
        print(f"Error: {str(e)}")
        return None
    
    return results

# Example usage
if __name__ == "__main__":
    trial_range = input("Enter trial range (e.g., '20-25'): ")
    results = calculate_average_bstrength(trial_range)
    
    if results:
        print("\nAverage of last 10 B_strength values:")
        for trial, avg in results.items():
            print(f"Trial {trial}: {avg:.6e}")
        
        # Calculate overall average
        overall_avg = np.mean(list(results.values()))
        print(f"\nOverall average: {overall_avg:.6e}")
